In [1]:
from model.dataloader import NinaProDatasetLoader, NearlabDatasetLoader
from model.nina_helper import *
import torch
import pandas as pd
from sklearn.utils import shuffle
from scipy import signal
import os
from model.utils import list_files
from model.model import CNet2D

import warnings

warnings.filterwarnings("ignore")

In [2]:
path_nearlab = "E:/Dennis_Bachelor/Bachelor_Thesis-main/src/data/nearlab/8features/person"
path_ninapro = "E:/Dennis_Bachelor/Bachelor_Thesis-main/src/data/ninapro/DB2/person"
results_ninapro = "E:/Dennis_Bachelor/Bachelor_Thesis-main/src/results/ninapro"
results_nearlab = "E:/Dennis_Bachelor/Bachelor_Thesis-main/src/results/nearlab"
versions = ["Softmax", "GLVQ", "GMLVQ"]
n_shots = [1, 5, 10]
prototypes_per_class = [1, 2, 4, 8]

# Nearlab

# Load in all Subject Data

In [ ]:
# Runs all models with all subjects expect one as training and one as testing
def run_model_all_subjects():
    for person in range(1, 12):
        train_paths = [x for x in list_files(path_nearlab + f"{person}", "csv")]
        test_paths = []
        for person2 in range(1, 12):
            if person2 != person:
                test_paths.append([train_paths.append(x) for x in list_files(path_nearlab + f"{person2}", "csv")])
        
        data = NearlabDatasetLoader(train_paths, test_paths)
        X_train, y_train, X_val, y_val, X_test, y_test = data.load_data()

        for version in versions:
            # Recommended Settings
            current_model = CNet2D(version=version, epochs=400, batch_size=128, learning_rate=0.0002)
            history = current_model.fit(X_train, y_train, 10, X_val, y_val)
            # Add accuracy of model to history
            history["accuracy"] = current_model.evaluate_model(X_test, y_test)
            # Checks if folder exists, if not creates it
            if not os.path.exists(results_nearlab + f"/person{person}/"):
                os.makedirs(results_nearlab + f"/person{person}/")
            # Save model
            current_model.save_model(results_nearlab + f"/person{person}/model_{version}_Subject_{person}.pt")
            # Save history
            current_model.save_history_csv(history, results_nearlab + f"/person{person}/history_{version}_Subject_{person}.csv")


In [9]:
# Runs all models with all hand-orientation combinations on each subject where 2 files are used for training and 1 for testing
def run_model_all_orientations(num_prototype_per_class):
    for person in range(1, 12):
        for orien in range(1, 4):
            all_files = list_files(path_nearlab + f"{person}", "csv")
            test_paths = [all_files[orien]]
            train_paths = [x for x in all_files if x not in test_paths]
            data = NearlabDatasetLoader(train_paths, test_paths)
            X_train, y_train, X_val, y_val, X_test, y_test = data.load_data()

            for version in versions:
                # Recommended Settings
                current_model = CNet2D(version=version, epochs=400, batch_size=128, learning_rate=0.0002, num_prototypes_per_class=num_prototype_per_class)
                history = current_model.fit(X_train, y_train, 10, X_val, y_val)
                # Add accuracy of model to history
                history["accuracy"] = current_model.evaluate_model(X_test, y_test)
                # Checks if folder exists, if not creates it
                if not os.path.exists(results_nearlab + f"/person{person}/"):
                    os.makedirs(results_nearlab + f"/person{person}/")
                # Save model
                current_model.save_model(results_nearlab + f"/person{person}/model_{version}_Orientation_{orien}_Prototypes_{num_prototype_per_class}.pt")
                # Save history
                current_model.save_history_csv(history, results_nearlab + f"/person{person}/history_{version}_Orientation_{orien}_Prototypes_{num_prototype_per_class}.csv")

In [10]:
# Runs k shot FSL on all subjects with 2 files used for training and 1 for testing
def run_k_shot_FSL(k_shots):
    for person in range(1, 12):
        all_files = list_files(path_nearlab + f"{person}", "csv")
        test_paths = [all_files[2]]
        train_paths = [x for x in all_files if x not in test_paths]
        data = NearlabDatasetLoader(train_paths, test_paths)
        X_train, y_train, X_val, y_val, X_test, y_test = data.load_data()
        # Select random class
        unique_classes = np.unique(y_train)
        random_class = np.random.choice(unique_classes)
        # Select all classes from X_train except the random class
        X_train_filtered = X_train[y_train != random_class]
        y_train_filtered = y_train[y_train != random_class]
        # Select all classes from X_val except the random class
        X_val = X_val[y_val != random_class]
        y_val = y_val[y_val != random_class]
        # Select k_shots samples from X_train
        few_shot_mask = y_test == random_class
        few_shot_indices = np.random.choice(np.where(few_shot_mask)[0], k_shots, replace=False)
        few_shot_X = X_test[few_shot_indices]
        
        for version in versions:
            # Recommended Settings from paper
            current_model = CNet2D(version=version, epochs=400, batch_size=128, learning_rate=0.0002)
            history = current_model.fit(X_train_filtered, y_train_filtered, 10, X_val, y_val)
            # Add accuracy of model to history
            current_model.add_new_class(few_shot_X)
            current_model.optimize_new_prototypes(few_shot_X, epochs=5)
            history["accuracy"] = current_model.evaluate_model(X_test, y_test)
            # Checks if folder exists, if not creates it
            if not os.path.exists(results_nearlab + f"/person{person}/"):
                os.makedirs(results_nearlab + f"/person{person}/")
            # Save model
            current_model.save_model(results_nearlab + f"/person{person}/model_{version}_K_Shot_{k_shots}.pt")
            # Save history
            current_model.save_history_csv(history, results_nearlab + f"/person{person}/history_{version}_K_Shot_{k_shots}.csv")

# Ninapro

In [3]:
folder_path = "/Users/dennisschielke/Desktop/Uni/Bachelor_Thesis/src/data/ninapro/DB2/person1/"

ninapro = NinaProDatasetLoader(folder_path, 1, 2, 512, 128)

X_train, y_train, X_test, y_test = ninapro.load_data(split_method="balanced", test_reps=2)

print(X_train.shape)

Shape of EMG data: (2553289, 12)


ValueError: Length of values (5106578) does not match length of index (2553289)

In [10]:
path_nearlab = "/Users/dennisschielke/Desktop/Uni/Bachelor_Thesis/src/data/nearlab/8features/person1"
file_paths_nearlab = list_files(path_nearlab, "csv")
version = "GMLVQ"
data = NearlabDatasetLoader(file_paths_nearlab[:2], file_paths_nearlab[2:])
X_train, y_train, X_test, y_test = data.load_data(split_method="repetition_wise")

print(X_train.shape)

torch.Size([4928, 10, 512])
